# Assignment 3
## Jupiter Notebook 1: Tidying Data


- Student Name: Karina Jonina
- Student ID: c00278440

- Module Name: Programming for Data Scientists
- Module Code: PROGC5201
- Module Leader: Paul Barry

- Course Name: 		Master of Science in Data Science
- Course Code:		 CW_KCDAR_M Y5

- Due Date: 10th January 2022 9:00am

In [ ]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re
from datetime import datetime
import numpy as np

In [ ]:
# gettting the CSV from the website
url = 'https://opendata-geohive.hub.arcgis.com/datasets/d8eb52d56273413b84b0187a4e9117be_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D.'

# read url csv
df = pd.read_csv(url)

In [ ]:
# Ensuring the Date is just date, and not datetime
df['Date'] = pd.to_datetime(df['Date']).dt.date
df.head()

In [ ]:
# examining columns and rows
df.shape

In [ ]:
# checking type for each column
df.info()

In [ ]:
# examining the names of the columns
df.columns

## Examining location
Below exploration showed that there are no unique locations in the dataset and therefore, redundant. 

In [ ]:
df['X'].unique()

In [ ]:
df['Y'].unique()

In [ ]:
df['CommunityTransmission'].unique()

In [ ]:
df['CloseContact'].unique()

In [ ]:
df['TravelAbroad'].unique()

### Dropping Columns (X and Y)

| Column | Reason |
|---|---|
| df['X'] | no unique values |
| df['Y'] | no unique values |
| df['StatisticsProfileDate'] | Very similar to df['Date']|
| df['CovidCasesConfirmed'] | Very similar to df['Total Confirmed Covid Cases'] |
| df['FID'] | very similar to Index in pandas |
| df['CommunityTransmission'] | no unique values |
| df['CloseContact'] | no unique values |
| df['TravelAbroad'] | no unique values |


In [ ]:
# Dropping irrelevant columns
df = df.drop(columns = ['X','Y','CovidCasesConfirmed','StatisticsProfileDate', 'FID', 
                        'CommunityTransmission', 'CloseContact','TravelAbroad'])

## Examining date
Below exploration showed that there are are unique dates in the dataset. 

In [ ]:
df['Date'].unique()

## Melting 

In [ ]:
# Melting
df = pd.melt(frame=df,id_vars = ["Date"], var_name = "Column", value_name = "Cases")

df.head(10)


### Extracting Age Range from values in 'Column'

In [ ]:
# creating new column 'Age Range' that is a copy of 'Column'
df['Age Range'] = df['Column']
df.tail()

In [ ]:
# replacing 'to' with '-'
df['Age Range'] = df['Age Range'].str.replace('to','-')

In [ ]:
# replacing 'to' with '-'
df['Age Range']= df['Age Range'].str.replace('up','+')

In [ ]:
# examining the new column
# some changes visible
df['Age Range']

In [ ]:
# replacing the rest, only aiming to have age range or NaN
df['Age Range']= df['Age Range'].replace("TotalConfirmedCovidCases", np.nan)
df['Age Range']= df['Age Range'].replace("ConfirmedCovidCases",np.nan)
df['Age Range']= df['Age Range'].replace("TotalCovidDeaths", np.nan)
df['Age Range']= df['Age Range'].replace("ConfirmedCovidDeaths",np.nan)
df['Age Range']= df['Age Range'].replace("HospitalisedCovidCases", np.nan)
df['Age Range']= df['Age Range'].replace("RequiringICUCovidCases", np.nan)
df['Age Range']= df['Age Range'].replace("HealthcareWorkersCovidCases", np.nan)
df['Age Range']= df['Age Range'].replace("ClustersNotified", np.nan)
df['Age Range']= df['Age Range'].str.replace("Aged", '')
df['Age Range']= df['Age Range'].str.replace("Hospitalised5", '1-4')
df['Age Range']= df['Age Range'].str.replace("1-4-14",'5-14')
df['Age Range']= df['Age Range'].str.replace("1-45-64",'55-64')
df['Age Range']= df['Age Range'].str.replace("Hospitalised", '')
df['Age Range']= df['Age Range'].replace("Median_Age", np.nan)
df['Age Range']= df['Age Range'].replace("Male", np.nan)
df['Age Range']= df['Age Range'].replace("Female", np.nan)
df['Age Range']= df['Age Range'].replace("Female", np.nan)
df['Age Range']= df['Age Range'].replace("Unknown",np.nan)

In [ ]:
df["Age Range"].unique()

In [ ]:
df["Column"].unique()

In [ ]:
# checking that 'HospitalisedAged55to64' with have 55-64 in 'Age Range'
df[df["Column"] =='HospitalisedAged55to64'].head()

In [ ]:
# checking that 'HospitalisedAged85up' with have 85+
df[df["Column"] =='HospitalisedAged85up'].head()

In [ ]:
# reseting the dataframe index
df = df.reset_index(drop = True)

In [ ]:
df.head()

In [ ]:
df.shape

### Dealing with ['Column'] column
I disliked the fact that the values are spelled out together with no space

In [ ]:
# Testing the regular expression
line = 'HospitalisedAged45to54'
match = re.findall('([A-Z][a-z]+)', line)
new_line = match[0] + " " + match[1] 
print(new_line)

In [ ]:
# using regular expression to put in a space
column_list = []
for i in  df["Column"]:
    match = re.findall('([A-Z][a-z]+)', i)
    if len(match) == 4:
        new_line = match[0] + ' ' + match[1] + ' ' + match[2] + ' ' + match[3]
    elif len(match) == 3:
        new_line = match[0] + ' ' + match[1] + ' ' + match[2]
    elif len(match) == 2:
        new_line = match[0] + ' ' + match[1] 
    else:
        new_line = match[0]

    column_list.append(new_line)

# examining the list
print(column_list)  

In [ ]:
# making the columnlist a pandas series
column_list = pd.Series(column_list)

# resetting the index on the pandas series
column_list = column_list.reset_index()

In [ ]:
# merging the new column to the dataset 
df = pd.merge(df, column_list, left_index = True, right_index = True)

In [ ]:
# renaming the columns (personal preference)
df = df.rename(columns={'Column': "old_column", 0 : "Column"})

In [ ]:
# changing the order of columns (personal preference)
df = df[[
    "Date",
    "old_column",
    "Column",
    "Age Range",
    "Cases"
]]

In [ ]:
df.head()

### Examining the dataset

In [ ]:
# selecting only where 'Column' contains "Hospitalised Aged"
hospitalised_df = df[df['Column'].str.contains("Hospitalised Aged")]

In [ ]:
# selecting only where Age Range is 65-74
age_range_65up = hospitalised_df[hospitalised_df['Age Range'] == '65-74']
age_range_65up.tail(10)

### Daily Cases
As pointed out by above table, hospitalised aged is based on accumulation, not daily cases.
After futher more exploration,  majority of columns are accumulated numbers, rather than daily cases

| Column | Explanation |
|---|---|
| Confirmed Covid Cases | Daily COVID-19 Cases |
| Total Confirmed Covid Cases | Accumulated COVID-19 Cases |
| Confirmed Covid Deaths | Daily COVID-19 Deaths |
| Median Age | Daily Median Age |

<br/>



##### Columns that require Daily Cases to be calculated
| Columns | Reason |
|---|---|
| Male | Accumulated COVID-19 Cases for Males |
| Female | Accumulated COVID-19 Cases for Females |
| Unknown | Accumulated COVID-19 Cases for Unknown |
| Healthcare Workers Covid Cases | Accumulated COVID-19 Cases for Healthcare Workers |
| Requiring ICU Covid Cases | Accumulated COVID-19 Cases that Requiring ICU Covid Cases |
| Hospitalised Covid Cases | Accumulated COVID-19 Cases that were Hospitalised |
| Clusters Notified | Accumulated Number of Clusters |
| Aged | Accumulated COVID-19 Cases for *each Age Range* |
| Hospitalised Aged | Accumulated COVID-19 Cases that were Hospitalised  for *each Age Range* |

In [ ]:
df['Column'].unique()

In [ ]:
# changing a value for the correct value
df['Column']= df['Column'].str.replace("Requiring Covid Cases", 'Requiring ICU Covid Cases')

In [ ]:
# creating a list with ALL values from Column 
column_list = ['Confirmed Covid Cases', 'Total Confirmed Covid Cases',
       'Confirmed Covid Deaths', 'Total Covid Deaths',
       'Hospitalised Covid Cases', 'Requiring ICU Covid Cases',
       'Healthcare Workers Covid Cases', 'Clusters Notified',
       'Hospitalised Aged', 'Male', 'Female', 'Unknown', 'Aged',
       'Median Age']
print(column_list)

In [ ]:
# Columns that have accumulated cases and no daily cases count 
accumulated = ['Male','Hospitalised Covid Cases', 
               'Healthcare Workers Covid Cases',  'Unknown', 'Female',
               'Median Age', 'Requiring Covid Cases', 'Clusters Notified']
print(accumulated)

In [ ]:
# creating a list with all age ranges
age_list = ['1-4', '5-14', '15-24', '25-34', '35-44', '45-54', '55-64', '65-74', '75-84', '85+']
print(age_list)

In [ ]:
# creating a loop where:
# IF Hospitalised Aged:
    # for age in age_list:
        # make a new dataframe for that age using correct dataset
        # calculate 'Daily Cases' with .diff(1) function
        # create the difference between daily cases (may be used later)
        # place this dataframe into a list 

        
# selecting only where 'Column' contains "Hospitalised Aged"
hospitalised_df = df[df['Column'].str.contains("Hospitalised Aged")]

# selecting only where 'Column' starts " Aged"
aged_df = df[df['Column'].str.startswith("Aged")]

# create a new list 
df_list = [] 


for i in column_list:
    if "Hospitalised Aged" == i:  
        for age in age_list: 
            
            # make a new dataframe for that age
            h_df = hospitalised_df[hospitalised_df['Age Range'].str.contains(age)] 
            
            # calculate 'Daily Cases' with .diff(1) function
            h_df['Daily Cases'] = h_df['Cases'].diff(1) 
            
            # append  this dataframe into a list 
            df_list.append(h_df) 
        
    elif "Aged" == i: 
        for age in age_list:
            
            # make a new dataframe for that age
            a_df = aged_df[aged_df['Age Range'].str.contains(age)]
            
            # calculate 'Daily Cases' with .diff(1) function
            a_df['Daily Cases'] = a_df['Cases'].diff(1)
            
            # append  this dataframe into a list 
            df_list.append(a_df)
    
    
#this is for 'Male','Hospitalised Covid Cases', 'Healthcare Workers Covid Cases',  'Unknown', 
# 'Female', 'Median Age', 'Requiring Covid Cases', 'Clusters Notified'
    elif  i in accumulated:
        n_df = df[df['Column'].str.contains(i)]
        
        # calculate 'Daily Cases' with .diff(1) function
        n_df['Daily Cases'] = n_df['Cases'].diff(1)
        

        # append  this dataframe into a list 
        df_list.append(n_df)
    
# this is for Confirmed Covid Cases', 'Total Confirmed Covid Cases', 
# 'Confirmed Covid Deaths', 'Total Covid Deaths',
# These columns are like we need them, however, there could be issues when concating if 'Daily Cases' and 
# 'Diff' are not there so NaN will be used for them.
    else:
         
        n_df = df[df['Column'].str.contains(i)]
        
        # calculate 'Daily Cases' with .diff(1) function
        n_df['Daily Cases'] = np.nan
        
        
        # append  this dataframe into a list 
        df_list.append(n_df) 

In [ ]:
# currently the dataframes are in the list
# by concating the list into dataframe
df =  pd.concat(df_list)

In [ ]:
# sorting data by date
df = df.sort_values(ascending = True, by = ["Date"])

# reseting the dataframe index
df = df.reset_index(drop=True)

In [ ]:
df.head(50)

In [ ]:
df.tail(50)

In [ ]:
# dropping old_column - only used for comparrisson to verify that it is correct
df = df.drop(columns = ['old_column'])

In [ ]:
# VERIFYING DAILY CASES HAS BEEN CORRECTLY IDENTIFIED

# selecting only where 'Column' contains "Hospitalised Aged"
hospitalised_df = df[df['Column'].str.contains("Hospitalised Aged")]
# selecting only where Age Range is 65-74
age_range_65up = hospitalised_df[hospitalised_df['Age Range'] == '65-74']
age_range_65up.tail(10)

### Saving to a new csv file

In [ ]:
df.to_csv(r"tidy_data_df.csv", index =  False)